# Roll: 1910776123 Assignment-2

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchinfo

# Load the pretrained VGG16 model

In [4]:
model = torchvision.models.vgg16(pretrained=True)
torchinfo.summary(model, input_size=(1, 3, 224, 224))

d:\books\course\2-2\CSE2252 Software Development Lab II\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
d:\books\course\2-2\CSE2252 Software Development Lab II\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

# VGG16 from scratch

In [13]:
class vgg16_like_model(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        
        #Feature Extraction Layer
        self.features = nn.Sequential(
            
            #1st Block of CNN
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=0),
            nn.ReLU(True), #inplace = True, so that value modified by the relu inside the tensor
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            #2nd Block of CNN
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            #3rd Block
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=0),
            nn.ReLU(True), 
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            #4th block
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            #5th Block         
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),             
        )
        
        #Pool Layer
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d(output_size=(7, 7))        
        
        #Classifier Layer -> Fully Connected or Dense Layer       
        self.classifier = nn.Sequential(
            nn.Linear(in_features=512*7*7, out_features=4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=output_size)
        )
    
    #Method for forward pass, must be implemented in every model
    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load the build vgg16

In [14]:
vgg16 = vgg16_like_model(output_size=1000)
torchinfo.summary(vgg16, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
vgg16_like_model                         [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 1, 1]            --
│    └─Conv2d: 2-1                       [1, 64, 222, 222]         1,792
│    └─ReLU: 2-2                         [1, 64, 222, 222]         --
│    └─Conv2d: 2-3                       [1, 64, 220, 220]         36,928
│    └─ReLU: 2-4                         [1, 64, 220, 220]         --
│    └─MaxPool2d: 2-5                    [1, 64, 110, 110]         --
│    └─Conv2d: 2-6                       [1, 128, 108, 108]        73,856
│    └─ReLU: 2-7                         [1, 128, 108, 108]        --
│    └─Conv2d: 2-8                       [1, 128, 106, 106]        147,584
│    └─ReLU: 2-9                         [1, 128, 106, 106]        --
│    └─MaxPool2d: 2-10                   [1, 128, 53, 53]          --
│    └─Conv2d: 2-11                      [1, 256, 51, 51]          29

# Transfer Weights of pretrained models into the newly built VGG16

In [22]:
# Pretrained model state dictionary (Collection of all the weights and biases)
model_state = model.state_dict()

# Build model state dictionary
vgg16_state = vgg16.state_dict()

# Update the model state dictionary with the pretrained model state dictionary
vgg16_state.update(model_state)

# Load the model state dictionary
vgg16.load_state_dict(vgg16_state)

# Check the weights of the first layer of both the models
model.features[0].weight, vgg16.features[0].weight

(Parameter containing:
 tensor([[[[-5.5373e-01,  1.4270e-01,  5.2896e-01],
           [-5.8312e-01,  3.5655e-01,  7.6566e-01],
           [-6.9022e-01, -4.8019e-02,  4.8409e-01]],
 
          [[ 1.7548e-01,  9.8630e-03, -8.1413e-02],
           [ 4.4089e-02, -7.0323e-02, -2.6035e-01],
           [ 1.3239e-01, -1.7279e-01, -1.3226e-01]],
 
          [[ 3.1303e-01, -1.6591e-01, -4.2752e-01],
           [ 4.7519e-01, -8.2677e-02, -4.8700e-01],
           [ 6.3203e-01,  1.9308e-02, -2.7753e-01]]],
 
 
         [[[ 2.3254e-01,  1.2666e-01,  1.8605e-01],
           [-4.2805e-01, -2.4349e-01,  2.4628e-01],
           [-2.5066e-01,  1.4177e-01, -5.4864e-03]],
 
          [[-1.4076e-01, -2.1903e-01,  1.5041e-01],
           [-8.4127e-01, -3.5176e-01,  5.6398e-01],
           [-2.4194e-01,  5.1928e-01,  5.3915e-01]],
 
          [[-3.1432e-01, -3.7048e-01, -1.3094e-01],
           [-4.7144e-01, -1.5503e-01,  3.4589e-01],
           [ 5.4384e-02,  5.8683e-01,  4.9580e-01]]],
 
 
         [[[ 1.77

# Build custom vgg16 of properties -
Architecture:
1. 4 Convolution Blocks (2, 2, 4, 4)
2. 2 Fully Connected Layers
3. Input shape (1, 3, 64, 64)
4. Output shape (1, 4)


In [33]:
# Architecture should be unique
class custom_vgg16(nn.Module):
    def __init__(self, output_size):
        super().__init__()
        self.features = nn.Sequential(
            
            #1st Block
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
            #2nd Block
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
            #3rd Block
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
            #4th Block
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
        )
        
        self.adaptive_max_pool = nn.AdaptiveMaxPool2d(output_size=(4, 4))
        self.classifier = nn.Sequential(
            nn.Linear(in_features=256*4*4, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=128, out_features=output_size)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_max_pool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load the custom vgg16 like model

In [34]:
custom_built_vgg16 = custom_vgg16(output_size=4)
custom_built_vgg16
torchinfo.summary(custom_built_vgg16, input_size=(1, 3, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
custom_vgg16                             [1, 4]                    --
├─Sequential: 1-1                        [1, 256, 4, 4]            --
│    └─Conv2d: 2-1                       [1, 64, 64, 64]           1,792
│    └─ReLU: 2-2                         [1, 64, 64, 64]           --
│    └─Conv2d: 2-3                       [1, 64, 64, 64]           36,928
│    └─ReLU: 2-4                         [1, 64, 64, 64]           --
│    └─AvgPool2d: 2-5                    [1, 64, 32, 32]           --
│    └─Conv2d: 2-6                       [1, 128, 32, 32]          73,856
│    └─ReLU: 2-7                         [1, 128, 32, 32]          --
│    └─Conv2d: 2-8                       [1, 128, 32, 32]          147,584
│    └─ReLU: 2-9                         [1, 128, 32, 32]          --
│    └─AvgPool2d: 2-10                   [1, 128, 16, 16]          --
│    └─Conv2d: 2-11                      [1, 128, 16, 16]          14